# AtLAST Predictions
Here, you can find another example of using this simulation tool for predictions. But now, in this scenario, we'll adjust all parameters to align with the AtLAST telescope design. AtLAST will have with a broad 2-degree field of view (FOV) and offers a 10" resolution at 150 GHz. This configuration provides a more comprehensive, high spatial dynamic range, ideal for observing phenomena such as the Sunyaev-Zeldovich effect in galaxy galaxys.

In [1]:
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np


KeyboardInterrupt



In [ ]:
from maria import Simulation
from maria import mappers

In [ ]:
from astropy.constants import c
from astropy import units as u
import time

In [ ]:
# Calculate observed frequency and channel width in GHz
def  skyfrq(line, z):
    return  line / (1 + z)

def widthfrq(frq, kms):
    return (kms/c_kms) * frq

In [ ]:
# Fits Image details
resval = 1024
simsize = 50 # kpc
CII_frq =   1900.537 # GHz

# Fits vals
redshifts = [6.5,       6.0,        3.0,        1.0,    0.5,        0.02,   0.01]
DL =        [64739.8,   58985.0,    25924.2,    6701.1, 2863.0,     87.5,   43.4] # Mpc
scale =     [5.580,     5.836,      7.855,      8.122,  6.169,      0.408,  0.206] # kpc/"
c_kms = 299792.458 # km s-1
channel = 50 # km s-1

frq = skyfrq(CII_frq, 1.0)

width = widthfrq(frq,channel)
print(f'Frequency:         {frq} GHz')
print(f'Channel:           {width} GHz, ({channel} km s-1)')

size                = simsize/ scale[3] / 3600 # deg
pixel_size          = size / resval # deg
pointing_center     = (10.0, 0.0) #RA and Dec in degrees
integration_time    = 1. * 60.0 #seconds
sample_rate         = 340. #Hz
scanning_speed      = 3/60 #deg/s
scanning_radius     = size*3/4 #deg
FOV                 = 0.002 # deg
Beam_size           = ((1.22*c/(frq*u.GHz).to(1/u.s)/(50 * u.m))*u.rad).to(u.arcsec)

print(f'Size:              { size*3600} arcsec')
print(f'Beam Size:         {Beam_size}')
print(f'Pixel Size:        {pixel_size*3500} arcsec' )
print()
print(f'Scan velocity:     {scanning_speed*3600} arcsec/s')
print(f'Field of View:     {FOV*3500} arcsec' )
print(f'Scanning Radius:   {scanning_radius*3500} arcsec' )

print('-----------------------')

In [ ]:
inputfile = "../../../maps/maria_Ponos_fits_CII_jy_1.0.fits"

noisy = False

outfile_map = '/Users/jvanmarr/Documents/Papers/mock_obs/output/galaxy_{}min_noisy{}_map.fits'.format(
    int(integration_time/60),
    str(noisy)
)

if noisy:
    atm_model            = 'single_layer'
    white_noise_level    = 0.
    pink_noise_level     = 0.
else:
    atm_model            = None
    white_noise_level    = 0.
    pink_noise_level     = 0.

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.constants import c

# - Input figure
cmap = "RdBu_r"

# - Input figure
hdu = fits.open(inputfile)
hdu[0].data = hdu[0].data
header = hdu[0].header

header['RESTFRQ'] = frq

header["CDELT1"] = pixel_size  # degree
header["CDELT2"] = pixel_size  # degree

header['CRPIX1'] = resval/2
header['CRPIX2'] = resval/2

header["CTYPE1"] = "RA---SIN"
header["CUNIT1"] = "deg     "
header["CTYPE2"] = "DEC--SIN"
header["CUNIT2"] = "deg     "
header['RADESYS']= "FK5     "

header["CRVAL1"] = pointing_center[0]
header["CRVAL2"] = pointing_center[1]
wcs_input = WCS(header, naxis=2)

In [ ]:
sky_l = SkyCoord((pointing_center[0] - FOV/4) * u.deg, (pointing_center[1] - FOV/4) * u.deg)
sky_u = SkyCoord((pointing_center[0] + FOV/4) * u.deg, (pointing_center[1] + FOV/4) * u.deg)

fig = plt.figure(dpi=256, tight_layout=False)
fig.set_size_inches(12, 5, forward=True)

# - Plot
ax = plt.subplot(1, 1, 1, projection=wcs_input)

im = ax.imshow(hdu[0].data * 1e3, cmap=cmap)
cbar = plt.colorbar(im, ax=ax, shrink=1.0)
cbar.set_label('mJy km/s/pixel')

ra, dec = ax.coords
ra.set_major_formatter("hh:mm:ss")
dec.set_major_formatter("dd:mm:ss")
ra.set_axislabel(r"RA [J2000]", size=11)
dec.set_axislabel(r"Dec [J2000]", size=11)
ra.set_separator(("h", "m"))

pixel_sky_l = wcs_input.world_to_pixel(sky_l)
pixel_sky_u = wcs_input.world_to_pixel(sky_u)
ax.axis(
    xmax=pixel_sky_l[0], ymin=pixel_sky_l[1], xmin=pixel_sky_u[0], ymax=pixel_sky_u[1]
)
plt.show()

### Fine-Tuning Simulation Parameters

To make predictions for AtLAST, several adjustments are required. Firstly, we need to change the pointing center. AtLAST is located near the APEX telescope in the southern hemisphere, so we set the pointing center to a Declination of -10. Additionally, we need to chance the atmospheric conditions. The defeault is set to mid-February at 6 am UT, which is an ideal time for observing with MUSTANG-2 on the GBT but not for AtLAST at Chajnantor. To achieve this, we modify the `start_time` key to August. This change also necessitates adjusting the Right Ascension (RA) of the pointing to ensure that the source remains above the horizon during the observation.

Furthermore, we overwrite the field of view to be 2 degrees and set the scan radius of the daisy scan to 1.3 degrees. We also adjust the detector bandwidth to 52 GHz with a total of 2000 detectors, and set a scan period of 120 seconds. 

Now, it's important to note that we haven't yet developed a mapper that can handle AtLAST's large FOV. Most of the contamination will likely be a common mode in the atmosphere, which needs proper Fourier filtering. This aspect is still a work in progress. Therefore, for the time being, we conduct noiseless observations by setting `atm_model = None`.

In [ ]:
#Alice's
sim = Simulation(

    # Mandatory minimal weather settings
    # ---------------------
    array="AtLAST",  # Array type
    pointing="daisy",  # Scanning strategy
    site="APEX",  # Site
 
    atm_model = atm_model,
    white_noise_level = white_noise_level,
    pink_noise_level  = pink_noise_level,
    
    # True sky input
    # ---------------------
    map_file     = inputfile,                # Input files must be a fits file.
    map_units    = 'Jy/pixel',               # Units of the input map in Kelvin Rayleigh-Jeans (K, default) or Jy/pixel
    map_res      = pixel_size,               # degree, overwrite header information
    map_center   = pointing_center,          # RA & Dec in degree

    # AtLAST Observational setup
    # ----------------------------
    integration_time = integration_time,  # Seconds
    sample_rate      = sample_rate,       # Hz
    scan_center      = pointing_center,   # Degrees
    start_time       = "2022-02-11T23:00:00",

    field_of_view    = FOV,
    dets             = {"f950": {"n": 200, "band_center": frq, "band_width": width}},
    scan_options     = {"speed": scanning_speed, "radius": scanning_radius, "petals": 2.11, "miss_factor": 0.05},
    pwv_rms_frac     = 0.005,
    layer_height     = 3e3,
)

In [ ]:
tod = sim.run()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=128)
map_extent = np.degrees(
[sim.map_sim.input_map.x_side.min(), sim.map_sim.input_map.x_side.max(), sim.map_sim.input_map.y_side.min(), sim.map_sim.input_map.y_side.max()]
) * 3600
map_im = ax.imshow(sim.map_sim.input_map.data[0], extent=map_extent)
if sim.map_sim.input_map.frame == "ra_dec":
    ax.set_xlabel("RA (arcsec.)")
    ax.set_ylabel("dec. (arcsec.)")
clb = fig.colorbar(mappable=map_im, shrink=0.8, aspect=32)

ax.axis(xmin = -1.8, xmax = 1.8, ymin = -1.8, ymax = 1.8)
clb.set_label(sim.map_sim.input_map.units)

In [ ]:
np.degrees(sim.map_sim.RA.mean()), sim.map_sim.pointing.scan_center

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(np.degrees(sim.map_sim.array.offset_x)*3600, np.degrees(sim.map_sim.array.offset_y)*3600)
plt.show()

In [ ]:
plt.plot(np.rad2deg(sim.map_sim.input_map.x_side)*3600, np.rad2deg(sim.map_sim.input_map.y_side)*3600)

In [ ]:
plt.figure()
for i in range(200):
    plt.plot((sim.map_sim.map_X[i,:1000])*3600, (sim.map_sim.map_Y[i,:1000])*3600, lw = 1)
# plt.plot((sim.map_sim.map_X[0,:1000])*3600, (sim.map_sim.map_Y[0,:1000])*3600, lw = 1)
# plt.plot((sim.map_sim.map_X[-1,:1000])*3600, (sim.map_sim.map_Y[-1,:1000])*3600, lw = 1)
plt.show()

In [ ]:
plt.scatter(sim.map_sim.map_X[0], sim.map_sim.map_Y[0])

### Visualizing the TOD Data

In this section, we present the same array and TOD visualizations as in the MUSTANG-2 case, but this time for AtLAST:

In [ ]:
np.unique(tod.data), tod.data

In [ ]:
# visualize scanning patern
# -----------------------
fig = plt.figure(dpi=256, tight_layout=True)
fig.set_size_inches(12, 5, forward=True)
fig.suptitle("Scanning strategy")

# - Plot
ax = plt.subplot(1,2,1)

ax.plot(np.degrees(tod.az), np.degrees(tod.el), lw=5e-1)
ax.set_xlabel("az (deg)"), ax.set_ylabel("el (deg)")

ax = plt.subplot(1, 2, 2, projection=wcs_input)
im = ax.imshow(hdu[0].data, cmap=cmap)

ra, dec = ax.coords
ra.set_major_formatter("hh:mm:ss")
dec.set_major_formatter("dd:mm:ss")
ra.set_axislabel(r"RA [J2000]", size=11)
dec.set_axislabel(r"Dec [J2000]", size=11)
ra.set_separator(("h", "m"))

sky = SkyCoord(np.degrees(tod.ra) * u.deg, np.degrees(tod.dec) * u.deg)
pixel_sky = wcs_input.world_to_pixel(sky)
ax.plot(pixel_sky[0], pixel_sky[1], lw=5e-1, alpha=0.5, c="yellow")
ax.set_xlabel("ra (deg)"), ax.set_ylabel("dec (deg)")
plt.show()

# visualize powerspectrum
# -----------------------
f, ps = sp.signal.periodogram(tod.data, fs=1/(tod.time[1] - tod.time[0]), window="tukey")

fig, axes = plt.subplots(1, 2, figsize=(10, 5), dpi=256, tight_layout=True)
fig.suptitle("TOD data")
axes[0].plot(f[1:], ps.mean(axis=0)[1:], label="Observation")
axes[0].plot(f[1:], f[1:] ** (-8 / 3), label="y = f^-(8/3)")
axes[0].set_xlabel(r"$\mathscr{f}$ [Hz]"), axes[0].set_ylabel("Power Spectra")
axes[0].loglog(), axes[0].legend()

for uib, uband in enumerate(np.unique(tod.dets.band)):
    band_mask = tod.dets.band == uband
    band_mean = tod.data[band_mask].mean(axis=0)
    axes[1].plot(tod.time - tod.time[0], band_mean, label=f"{uband} mean")

axes[1].set_xlabel("time (seconds)"), axes[1].set_ylabel(r"temperature ($K_{RJ}$)")
axes[1].legend()

plt.show()

# Test Scanning

In [ ]:
def f_lambda(f = frq, D = 50):
    return ((1.22 * (c/(f * u.GHz).to(1/u.s)).to(u.meter) / (D * u.m)) * u.rad).to(u.degree)

In [ ]:
sky_l = SkyCoord((pointing_center[0] - 1/3600) * u.deg, (pointing_center[1] - 1/3600) * u.deg)
sky_u = SkyCoord((pointing_center[0] + 1/3600) * u.deg, (pointing_center[1] + 1/3600) * u.deg)

pixel_sky_l = wcs_input.world_to_pixel(sky_l)
pixel_sky_u = wcs_input.world_to_pixel(sky_u)

fig = plt.figure(dpi=512, )
fig.set_size_inches(6, 5, forward=True)
ax = plt.subplot(1, 1, 1, projection=wcs_input)
im = ax.imshow(hdu[0].data, cmap=cmap)

ra, dec = ax.coords
ra.set_major_formatter("hh:mm:ss")
dec.set_major_formatter("dd:mm:ss")
ra.set_axislabel(r"RA [J2000]", size=11)
dec.set_axislabel(r"Dec [J2000]", size=11)
ra.set_separator(("h", "m"))

sky = SkyCoord(np.degrees(tod.ra) * u.deg, np.degrees(tod.dec) * u.deg)
pixel_sky = wcs_input.world_to_pixel(sky)

ax.plot(pixel_sky[0], pixel_sky[1], lw=5e-1, alpha=0.5, c="yellow")
ax.set_xlabel("ra (deg)"), ax.set_ylabel("dec (deg)")

ax.axis(xmin = pixel_sky_l[0], ymin = pixel_sky_l[1], xmax = pixel_sky_u[0], ymax = pixel_sky_u[1])
plt.show()

In [ ]:
sim.array.plot_dets()

In [ ]:


plt.figure(figsize=(4.5, 4.5),dpi=256)

plt.plot(
    np.degrees(tod.ra - tod.ra.mean())*3600,
    np.degrees(tod.dec - tod.dec.mean())*3600,
    lw=5e-1,
    alpha=0.5,
)

plt.scatter(
    np.degrees(sim.array.offset_x)*3600,
    np.degrees(sim.array.offset_y)*3600,
    label=f"{uband} mean",
    s=2,
    c="orange",
)

# plt.axis(xmin = -1, xmax = 1, ymin = -1, ymax = 1)
    
plt.xlabel(r"$\theta_x$ offset (arcsec)")
plt.ylabel(r"$\theta_y$ offset (arcsec)")
plt.show()

In [ ]:
plt.figure(figsize=(4.5, 4.5),dpi=256)
plt.grid(lw = 0.1)

xxs = np.degrees(sim.array.offset_x)*u.degree/f_lambda()
yys = np.degrees(sim.array.offset_y)*u.degree/f_lambda() 

plt.scatter(
    xxs,
    yys,
    s=1, c="orange")

plt.axis(xmin = -1, xmax = 1, ymin = -1, ymax = 1)
    
plt.xlabel(r"$\theta_x$ offset (f-$\lambda$)")
plt.ylabel(r"$\theta_y$ offset (f-$\lambda$)")
plt.show()

## Map-Making

As previously mentioned, we must disable Fourier filtering. Additionally, we have adjusted the height and width of the map to match the realistic AtLAST field of view, which spans several degrees.

In [ ]:
Beam_size

In [ ]:
s1 = time.time()

mapper = mappers.BinMapper(
    map_height=np.radians(size),
    map_width=np.radians(size),
    map_res=np.radians(4*pixel_size),
    map_filter=True,
    n_modes_to_remove=1,
    map_smooth = Beam_size.value #arcsec
)
mapper.add_tods(tod)
mapper.run()

s2 = time.time()
print(s2-s1)

In [ ]:
# f, ps = sp.signal.periodogram(tod.data, fs=1/(tod.time[1] - tod.time[0]), window="tukey")

# fig, axes = plt.subplots(1, 2, figsize=(10, 5), dpi=256, tight_layout=True)
# fig.suptitle("TOD data")
# axes[0].plot(f[1:], ps.mean(axis=0)[1:], label="Observation")
# axes[0].plot(f[1:], f[1:] ** (-8 / 3), label="y = f^-(8/3)")
# axes[0].set_xlabel(r"$\mathscr{f}$ [Hz]"), axes[0].set_ylabel("Power Spectra")
# axes[0].loglog(), axes[0].legend()

# for uib, uband in enumerate(np.unique(tod.dets.band)):
#     band_mask = tod.dets.band == uband
#     band_mean = tod.data[band_mask].mean(axis=0)
#     axes[1].plot(tod.time - tod.time[0], band_mean, label=f"{uband} mean")

# axes[1].set_xlabel("time (seconds)"), axes[1].set_ylabel(r"temperature ($K_{RJ}$)")
# axes[1].legend()

# plt.show()

In [ ]:
fig, axes = plt.subplots(
    1,
    2,
    figsize=(10, 4.5),
    dpi=256,
    tight_layout=True,
    gridspec_kw={"width_ratios": [1, 1.5]},
)
fig.suptitle("Detector setup for one band")

for uband in sim.array.ubands:
    band_mask = sim.array.dets.band == uband

    axes[0].plot(
        60 * np.degrees(tod.ra - tod.ra.mean()),
        60 * np.degrees(tod.dec - tod.dec.mean()),
        lw=5e-1,
        alpha=0.5,
    )
    
    axes[0].scatter(
        60 * np.degrees(sim.array.offset_x[band_mask]),
        60 * np.degrees(sim.array.offset_y[band_mask]),
        label=f"{uband} mean",
        lw=5e-1,
        c="orange",
    )
    
    break
axes[0].set_xlabel(r"$\theta_x$ offset (arcminutes)"), axes[0].set_ylabel(
    r"$\theta_y$ offset (arcminutes)"
)
axes[0].legend()

xs, ys = np.meshgrid(
    60 * np.rad2deg((mapper.x_bins[1:] + mapper.x_bins[:-1]) / 2),
    60 * np.rad2deg((mapper.y_bins[1:] + mapper.y_bins[:-1]) / 2),
)

im = axes[1].pcolormesh(
    xs,
    ys,
    mapper.map_cnts[tod.dets.band[0]],
    label="Photon counts in band " + tod.dets.band[0],
)

axes[1].set_xlabel(r"$\theta_x$ (arcmin)"), axes[1].set_ylabel(r"$\theta_y$ (arcmin)")
cbar = plt.colorbar(im, ax=axes[1])
cbar.set_label("Counts")
plt.show()

## Save & Visualize

In [ ]:
mapper.save_maps(outfile_map)

In [ ]:
sky_l = SkyCoord((pointing_center[0] - 2) * u.deg, (pointing_center[1] - 2) * u.deg)
sky_u = SkyCoord((pointing_center[0] + 2) * u.deg, (pointing_center[1] + 2) * u.deg)

fig = plt.figure(dpi=256, tight_layout=False)
fig.set_size_inches(12, 5, forward=True)

# - Plot
ax = plt.subplot(1, 1, 1, projection=wcs_input)
ax.set_title("True Sky")
im = ax.imshow(hdu[0].data * 1e3, cmap=cmap)
cbar = plt.colorbar(im, ax=ax, shrink=1.0)

ra, dec = ax.coords
ra.set_major_formatter("hh:mm:ss")
dec.set_major_formatter("dd:mm:ss")
ra.set_axislabel(r"RA [J2000]", size=11)
dec.set_axislabel(r"Dec [J2000]", size=11)
ra.set_separator(("h", "m"))

pixel_sky_l = wcs_input.world_to_pixel(sky_l)
pixel_sky_u = wcs_input.world_to_pixel(sky_u)
ax.axis(
    xmax=pixel_sky_l[0], ymin=pixel_sky_l[1], xmin=pixel_sky_u[0], ymax=pixel_sky_u[1]
)
plt.show()


# - Plot Mock observation
outputfile = outfile_map

hdu_out = fits.open(outputfile)
wcs_output = WCS(hdu_out[0].header, naxis=2)

# - Plot
fig = plt.figure(dpi=256, tight_layout=False)
fig.set_size_inches(6, 4, forward=True)

ax = plt.subplot(1, 1, 1, projection=wcs_output)
im = ax.imshow(hdu_out[0].data[0]*1e3, cmap=cmap, vmin=-0.5, vmax=0.5)
# im = ax.imshow(hdu_out[0].data[0]/(mapper.map_res**2)/1e6, cmap=cmap, vmin=-0.1, vmax=0.1)

cbar = plt.colorbar(im, ax=ax, shrink=1.0)
cbar.set_label(r"$I_\nu$ $[mK_{RJ}]$")
# cbar.set_label(r"S$_{93~\rm GHz}$ [MJy/sr]")

ra, dec = ax.coords
ra.set_major_formatter("hh:mm:ss")
dec.set_major_formatter("dd:mm:ss")
ra.set_axislabel(r"RA [J2000]", size=11)
dec.set_axislabel(r"Dec [J2000]", size=11)
ra.set_separator(("h", "m"))


sky_l = SkyCoord((pointing_center[0] - 2) * u.deg, (pointing_center[1] - 2) * u.deg)
sky_u = SkyCoord((pointing_center[0] + 2) * u.deg, (pointing_center[1] + 2) * u.deg)

pixel_sky_l = wcs_output.world_to_pixel(sky_l)
pixel_sky_u = wcs_output.world_to_pixel(sky_u)




# ax.axis(xmin = pixel_sky_l[0], ymin = pixel_sky_l[1], xmax = pixel_sky_u[0], ymax = pixel_sky_u[1])
# plt.savefig("/Users/jvanmarr/Documents/Papers/mock_obs/paper_plots/AtLAST_noiesemap.pdf")
plt.show()

## Fidelity Test

To gain insight into how the scanning strategy influences the quality of the observation, we've created a simple fidelity map. This map illustrates how modifying the scanning strategy impacts the recovery of your science. In particular, the sampling rate and scanning period have a significant influence on the image's fidelity. We are continually working on optimizing the scanning strategy for AtLAST, but you are welcome to experiment with these values yourself to observe their effect on the fidelity map.

In [ ]:
# from reproject import reproject_interp

# sim_projected, footprint = reproject_interp(
#     (hdu[0].data, wcs_input), wcs_output, shape_out=hdu_out[0].data[0].shape
# )

In [ ]:
# fig = plt.figure(dpi=256, tight_layout=False)
# fig.set_size_inches(12, 5, forward=True)

# # - Plot
# ax = plt.subplot(1, 1, 1, projection=wcs_output)
# ax.set_title("Weighted Ratio")
# im = ax.imshow((sim_projected / hdu_out[0].data[0]) - 1, cmap=cmap, vmin=-2, vmax=5)
# cbar = plt.colorbar(im, ax=ax, shrink=1.0)
# # cbar.set_label('mock/sim - 1')

# ra, dec = ax.coords
# ra.set_major_formatter("hh:mm:ss")
# dec.set_major_formatter("dd:mm:ss")
# ra.set_axislabel(r"RA [J2000]", size=11)
# dec.set_axislabel(r"Dec [J2000]", size=11)
# ra.set_separator(("h", "m"))

# pixel_sky_l = wcs_output.world_to_pixel(sky_l)
# pixel_sky_u = wcs_output.world_to_pixel(sky_u)
# ax.axis(
#     xmax=pixel_sky_l[0], ymin=pixel_sky_l[1], xmin=pixel_sky_u[0], ymax=pixel_sky_u[1]
# )

# plt.show()